In [4]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Load NTA polygons
nta = pd.read_csv("NYC_NTAs.csv")

# Convert WKT → geometry
nta["geometry"] = nta["the_geom"].apply(wkt.loads)

gdf_nta = gpd.GeoDataFrame(
    nta,
    geometry="geometry",
    crs="EPSG:4326"
)


In [24]:
citibike_df = pd.read_csv('202601-citibike-tripdata_2.csv')

In [25]:
# Extract unique start stations
start_stations = citibike_df[["start_station_id", "start_station_name", "start_lat", "start_lng"]].drop_duplicates()

# Extract unique end stations
end_stations = citibike_df[["end_station_id", "end_station_name", "end_lat", "end_lng"]].drop_duplicates()

# Rename end columns to match start columns
end_stations = end_stations.rename(columns={
    "end_station_id": "station_id",
    "end_station_name": "station_name",
    "end_lat": "lat",
    "end_lng": "lng"
})

start_stations = start_stations.rename(columns={
    "start_station_id": "station_id",
    "start_station_name": "station_name",
    "start_lat": "lat",
    "start_lng": "lng"
})

# Combine and dedupe
stations = pd.concat([start_stations, end_stations], ignore_index=True).drop_duplicates("station_id")

gdf_stations = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations.lng, stations.lat),
    crs="EPSG:4326"
)

stations_with_nta = gpd.sjoin(
    gdf_stations,
    gdf_nta[["NTA2020", "NTAName", "BoroName", "geometry"]],
    how="left",
    predicate="within"
)


In [26]:
stations_with_bergen = stations_with_nta[
    stations["station_name"].str.contains("bergen", case=False, na=False)
]

print(stations_with_bergen)


     station_id                    station_name        lat        lng  \
221     4281.08        Bergen St & Flatbush Ave  40.680945 -73.975673   
264     4190.06           Grand Ave & Bergen St  40.678045 -73.962408   
291     4072.02         Bergen St & Buffalo Ave  40.674700 -73.925120   
374     4066.15         Bedford Ave & Bergen St  40.676368 -73.952918   
475     4446.01            Bergen St & Smith St  40.686744 -73.990632   
563     4322.06               Bergen St & 4 Ave  40.682564 -73.979898   
775     4404.10             Bond St & Bergen St  40.684967 -73.986208   
789     4064.03        Bergen St & Saratoga Ave  40.674230 -73.916560   
790     4048.03            Bergen St & Troy Ave  40.675240 -73.935960   
791     4157.10      Bergen St & Vanderbilt Ave  40.679439 -73.968044   
1273    4089.01        Bergen St & Kingston Ave  40.675600 -73.941500   
1326    4055.04  Bergen St & Mother Gaston Blvd  40.673740 -73.908520   
1778    7858.02           Bergen Ave & E 152 St  40

In [27]:
# Merge start NTA
citibike_df = citibike_df.merge(
    stations_with_nta[["station_id", "NTAName", "BoroName"]].rename(columns={"station_id": "start_station_id", "NTAName": "start_NTA", "BoroName": "start_Boro"}),
    on="start_station_id",
    how="left"
)

# Merge end NTA
citibike_df = citibike_df.merge(
    stations_with_nta[["station_id", "NTAName", "BoroName"]].rename(columns={"station_id": "end_station_id", "NTAName": "end_NTA", "BoroName": "end_Boro"}),
    on="end_station_id",
    how="left"
)

citibike_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_NTA,start_Boro,end_NTA,end_Boro
0,0179695DC1F74E4E,classic_bike,2026-01-09 15:39:44.131,2026-01-09 15:44:27.970,Adam Clayton Powell Blvd & W 115 St,7643.18,St Nicholas Ave & Manhattan Ave,7723.01,40.802535,-73.953242,40.809725,-73.953149,member,Harlem (South),Manhattan,Harlem (South),Manhattan
1,5D0645202DDFA256,electric_bike,2026-01-09 10:14:39.226,2026-01-09 10:18:56.254,Carlton Ave & Dean St,4199.12,3 Ave & Schermerhorn St,4437.01,40.680974,-73.971010,40.686832,-73.979677,member,Prospect Heights,Brooklyn,Downtown Brooklyn-DUMBO-Boerum Hill,Brooklyn
2,4572C883BF0165E4,classic_bike,2026-01-14 18:44:00.839,2026-01-14 18:52:54.180,Broadway & Morris St,5033.01,Lispenard St & Broadway,5391.06,40.705945,-74.013219,40.719392,-74.002472,member,Financial District-Battery Park City,Manhattan,Tribeca-Civic Center,Manhattan
3,3575D616D0179F57,electric_bike,2026-01-06 05:57:18.257,2026-01-06 06:15:40.651,Van Brunt St & Wolcott St,4095.03,Vesey St & Church St,5216.06,40.677343,-74.012751,40.712220,-74.010472,member,Carroll Gardens-Cobble Hill-Gowanus-Red Hook,Brooklyn,Financial District-Battery Park City,Manhattan
4,895D2DC8F8BA4177,electric_bike,2026-01-02 16:19:07.373,2026-01-02 16:24:19.447,W 33 St & 10 Ave,6492.04,6 Ave & W 33 St,6364.07,40.753773,-73.999376,40.749013,-73.988484,member,Chelsea-Hudson Yards,Manhattan,Midtown South-Flatiron-Union Square,Manhattan


In [28]:
top_start_ntas = (
    citibike_df
    .groupby("start_NTA")
    .size()
    .reset_index(name="n_rides_start")
    .sort_values("n_rides_start", ascending=False)
)
print(top_start_ntas.head(15))

top_end_ntas = (
    citibike_df
    .groupby("end_NTA")
    .size()
    .reset_index(name="n_rides_end")
    .sort_values("n_rides_end", ascending=False)
)
print(top_end_ntas.head(15))

                                        start_NTA  n_rides_start
19                           Chelsea-Hudson Yards          57335
38                                   East Village          43867
74                           Midtown-Times Square          40308
73            Midtown South-Flatiron-Union Square          36008
118                                  Williamsburg          32358
81                           Murray Hill-Kips Bay          30694
55                                 Hell's Kitchen          25707
51                              Greenwich Village          23631
117                                  West Village          21670
69                                Lower East Side          21035
93                SoHo-Little Italy-Hudson Square          18987
111                Upper West Side-Lincoln Square          17949
17   Carroll Gardens-Cobble Hill-Gowanus-Red Hook          17641
50                                     Greenpoint          17570
37                       

In [33]:
nta_flows = (
    citibike_df
    .groupby(["start_NTA", "start_Boro", "end_NTA", "end_Boro"])
    .size()
    .reset_index(name="n_rides")
    .sort_values("n_rides", ascending=False)
)
nta_flows_not_same = nta_flows[(nta_flows["start_Boro"] != nta_flows["end_Boro"]) & ((nta_flows["start_Boro"] == "Manhattan") | (nta_flows["end_Boro"] == "Manhattan"))]
nta_flows_not_same.head(30)

,start_NTA,start_Boro,end_NTA,end_Boro,n_rides
5581,Williamsburg,Brooklyn,Lower East Side,Manhattan,1085
3186,Lower East Side,Manhattan,Williamsburg,Brooklyn,960
5596,Williamsburg,Brooklyn,SoHo-Little Italy-Hudson Square,Manhattan,637
4424,SoHo-Little Italy-Hudson Square,Manhattan,Williamsburg,Brooklyn,548
5563,Williamsburg,Brooklyn,East Village,Manhattan,537
1814,East Village,Manhattan,Williamsburg,Brooklyn,437
5566,Williamsburg,Brooklyn,Financial District-Battery Park City,Manhattan,340
5554,Williamsburg,Brooklyn,Chinatown-Two Bridges,Manhattan,335
4808,Tribeca-Civic Center,Manhattan,Downtown Brooklyn-DUMBO-Boerum Hill,Brooklyn,325
983,Chinatown-Two Bridges,Manhattan,Williamsburg,Brooklyn,317
